In [ ]:
!pip install langchain

In [ ]:
!pip install blosc2==2.0.0 cython>=0.29.21
!pip install fsspec==2023.10.0
!pip install black>=22.3.0

In [ ]:
!pip install langchainhub
!pip install langchain-openai
!pip install langchain_community

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
#이 파일과 같은 위치에 있는 .env 파일에 해당 키를 저장해주세요
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
llm = ChatOpenAI(openai_api_key="OPENAI_API_KEY")
llm = ChatOpenAI()

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [5]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [6]:
examples = [
    {
        "input": "안녕하세요, 개발자 직군에 지원한 홍길동입니다.",
        "output": "개발자 직군에 지원한 홍길동님 안녕하세요? 1번 면접 질문입니다. 지원동기가 무엇인가요?",
        "input": "관련 답변",
        "output": "2번 면접 질문입니다. AWS 서비스 중 LLM 기반 AI 면접 모델 개발에 가장 적합하다고 생각하는 서비스는 무엇이며, 그 이유는 무엇인가요?",
        "input": "관련 답변",
        "output": "허깅페이스 모델의 성능 향상을 위해 어떤 노력을 기울였나요?",
        "input": "관련 답변",
        "output": "Airflow를 사용하여 LLM 기반 AI 면접 모델 개발 과정을 자동화한 경험이 있다면 구체적으로 설명해주세요.",
        "input": "관련 답변",
        "output": "5번 면접 질문입니다. TTS 및 STT 기술을 활용하여 LLM 기반 AI 면접 모델의 사용자 경험을 어떻게 향상시킬 수 있나요?",
        "input": "관련 답변",
        "output": "6번 면접 질문입니다.",
        "input": "관련 답변",
        "output": "7번 면접 질문입니다.",
        "input": "관련 답변",
        "output": "8번 면접 질문입니다.",
        "input": "관련 답변",
        "output": "9번 면접 질문입니다.",
        "input": "관련 답변",
        "output": "10번 면접 질문입니다.",
        "input": "관련 답변",
        "output": "면접이 종료되었습니다. 이제 해주신 답변에 대한 피드백을 드리겠습니다. 1번 질문에 대한 답변의 경우..... ",
        
    },
    {
        "input": "안녕하세요, 프론트엔드 직군에 지원한 홍길동입니다.",
        "output": "프론트엔드 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, 백엔드 직군에 지원한 홍길동입니다.",
        "output": "백엔드 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, 데이터베이스 직군에 지원한 홍길동입니다.",
        "output": "데이터베이스 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, AI개발 직군에 지원한 홍길동입니다.",
        "output": "AI개발 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, 생산 직군에 지원한 홍길동입니다.",
        "output": "생산 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, 건설 직군에 지원한 홍길동입니다.",
        "output": "건설 직군에 지원한 홍길동님 안녕하세요?",
    },
    {
        "input": "안녕하세요, 의료 직군에 지원한 홍길동입니다.",
        "output": "의료 직군에 지원한 홍길동님 안녕하세요?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [15]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2, #example을 선정 갯수
)

# input 내용과 가장 유사한 example을 선정
example_selector.select_examples({"input": "안녕하세요, 개발자 직군에 지원한 홍길동입니다."})

[{'input': '안녕하세요, AI개발 직군에 지원한 홍길동입니다.',
  'output': 'AI개발 직군에 지원한 홍길동님 안녕하세요?'},
 {'input': '안녕하세요, 건설 직군에 지원한 홍길동입니다.', 'output': '건설 직군에 지원한 홍길동님 안녕하세요?'}]

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage
직무명 = "AI개발자"
자기소개서 = "안녕하세요, 저는 컴퓨터 공학을 전공하고 AI 개발에 깊은 관심을 가지고 있는 홍길동입니다. 대학 시절부터 머신러닝 프로젝트에 참여하며 기술적 역량을 쌓아왔습니다. 특히, 자연어 처리(NLP)와 컴퓨터 비전 분야에서 실제 문제를 해결하기 위한 여러 프로젝트를 진행했습니다. 예를 들어, 소셜 미디어 데이터를 분석하여 감정을 분류하는 모델을 개발하여, 데이터로부터 유의미한 인사이트를 추출하는 데 기여했습니다. 또한, 최신 AI 기술 동향을 끊임없이 학습하며, 오픈소스 프로젝트에 기여한 경험도 있습니다. 이러한 경험을 통해 협업과 커뮤니케이션 능력뿐만 아니라, 문제 해결 능력을 향상시킬 수 있었습니다. 저는 AI 개발자로서 이러한 기술적 배경과 열정을 바탕으로贵公司에서 발전 가능성을 모색하고 싶습니다. AI 기술이 사회에 긍정적인 변화를 가져올 수 있도록 기여하고 싶습니다. 감사합니다."
필수_질문 = "지원동기가 무엇인가요?"

    
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",f"나는 지원자 역할을 하고, 당신은 면접관 역할을 한다. 우리는 {직무명} 포지션에 대한 면접을 진행한다. 면접 과정은 오로지 한국어로만 진행된다. 지원자의 자기소개서는 {자기소개서}이다. 지원자의 자기소개서를 기반으로 한 질문들이 포함될 것이며, '지원 동기', '기술적 능력', '팀워크 및 협업' 카테고리에서 최소 하나의 질문을 하고, 사용자가 설정한 {필수_질문}도 반드시 질문한다. 지원자의 자기소개서에서 주요한 경험, 성과, 또는 기술에 대해 질문하여, 지원자가 해당 포지션에 어떻게 기여할 수 있는지 구체적으로 이해한다. 각 카테고리와 자기소개서에서 얻은 정보에 기반한 질문을 제시하고, 나의 답변을 기다린 후 다음 질문으로 넘어간다. 설명이나 다른 형태의 대화는 제공하지 않는다. '안녕하세요'라고 말하면 면접을 시작한다. 질문 카운터를 설정하고, 각 질문 후에 카운터를 1씩 증가시킨다. 질문이 총 10개 제시되었을 때, 혹은 지원자가 '면접 종료'라고 말했을 때만 면접을 종료한다. 만약 10개의 질문이 완료되기 전에 '면접 종료'라는 말이 없었다면, 계속해서 질문을 이어간다. 면접 종료 시, 지원자에게 제공할 피드백을 준비한다"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
llm = ChatOpenAI()
chain = final_prompt | llm

In [11]:
# 대화 반복
# 사용자로부터 직종 정보 입력받기
user_job_title = input("지원하는 직군을 입력해주세요: ")

# 인사말을 구성하는 변수 설정
greeting_message = f"안녕하세요, {user_job_title} 직군에 지원한 홍길동입니다."

# 대화의 첫 입력으로 인사말 사용
current_input = greeting_message

# 동적 예시 선택기를 사용하여 관련 예시를 출력
print(example_selector.select_examples({"input": greeting_message}))
while True:
    # 체인에 인사말을 입력으로 전달
    response = chain.invoke({"input": current_input})
    print(response.content)  # 챗봇의 응답 출력

    # 사용자 입력 받기
    user_input = input("Your response: ")
    if user_input == "면접 종료":  # 면접 종료 조건
        break
    # 사용자의 답변을 다음 챗봇 입력으로 설정
    current_input = user_input
    # 사용자 대답을 메시지 리스트에 추가하고 반복
    final_prompt.append(HumanMessage(content=user_input))


지원하는 직군을 입력해주세요: 개발자
[{'input': '안녕하세요, AI개발 직군에 지원한 홍길동입니다.', 'output': 'AI개발 직군에 지원한 홍길동님 안녕하세요?'}, {'input': '안녕하세요, 건설 직군에 지원한 홍길동입니다.', 'output': '건설 직군에 지원한 홍길동님 안녕하세요?'}]
안녕하세요, 홍길동님. 저희는 AI 개발자 포지션에 대한 면접을 진행하려고 합니다. 제가 면접관입니다. 그럼 첫 질문을 시작하겠습니다. 홍길동님의 자기소개서를 토대로, 자연어 처리(NLP)와 컴퓨터 비전 분야에서의 프로젝트 경험에 대해 좀 더 자세히 설명해주실 수 있을까요?
Your response: 네 주로 뉴스 데이터를 통해 카테고리를 예측하는 프로젝트와 객체 인식을 통해 물건을 분류하는 프로젝트를 하였습니다.
홍길동님, 뉴스 데이터를 활용하여 카테고리를 예측하는 프로젝트와 객체 인식을 통해 물건을 분류하는 프로젝트에 대해 좀 더 자세히 알고 싶습니다. 

1. 뉴스 데이터를 활용한 카테고리 예측 프로젝트에서는 어떤 알고리즘을 주로 사용했나요?
2. 객체 인식을 통해 물건을 분류하는 프로젝트에서는 어떤 종류의 물건을 분류했고, 어떤 성과를 이뤘나요?
Your response: 뉴스 데이터 기반 프로젝트에 경우 Bert 모델을 사용해 뉴스 데이터를 파인 튜닝하는 방식으로 분류 모델을 만들었습니다. 객체 인식에 경우 주로 편의점에서 판매하는 컵라면, 과자를 분류하였고, 이를 실제 작동하는 운반 기계에 접목시켜 자동 물품 진열기를 졸업작품으로 출품하였고, 산업기술대전 총장상을 수상하였습니다.
뉴스 데이터를 통한 카테고리 분류 모델과 객체 인식을 통한 물건 분류 프로젝트는 매우 흥미로운 경험입니다. 먼저, Bert 모델을 활용한 뉴스 데이터 분류 모델에 대해 궁금한 점이 있습니다. Bert 모델을 선택한 이유와 모델 성능 향상을 위해 어떤 노력을 기울였는지 설명해주실 수 있을까요?


KeyboardInterrupt: Interrupted by user